Congratulations on joining **TrueSound Research** — where cutting-edge machine learning meets the future of music. Our clients are the best in the industry, and now ***you*** are part of the team shaping what comes next.

**Your Task for Today:**

Build a music generator that creates one chunk of music at a time — each note, beat, or phrase predicting the next — in true autoregressive style.

The world is listening. Time to create something unforgettable.

***Let's make some noise.***

In [ ]:
import torch
import torchaudio
import matplotlib.pyplot as plt
from dataclasses import dataclass
import typing as T
import numpy as np
import io
import numpy as np
from scipy.io import wavfile

!pip install pydub
import pydub

In [ ]:
# @title Utility functions (run this cell)

@dataclass(frozen=True)
class SpectrogramParams:

    # Whether the audio is stereo or mono
    stereo: bool = False

    # FFT parameters
    sample_rate: int = 44100
    step_size_ms: int = 10
    window_duration_ms: int = 100
    padded_duration_ms: int = 400

    # Mel scale parameters
    num_frequencies: int = 512
    min_frequency: int = 0
    max_frequency: int = 10000  # TODO (Ema): probably better to use 20 to 20,000
    mel_scale_norm: T.Optional[str] = None
    mel_scale_type: str = "htk"

    # Griffin Lim parameters
    num_griffin_lim_iters: int = 32

    # Image parameterization
    power_for_image: float = 0.25

    @property
    def n_fft(self) -> int:
        """
        The number of samples in each STFT window, with padding.
        """
        return int(self.padded_duration_ms / 1000.0 * self.sample_rate)

    @property
    def win_length(self) -> int:
        """
        The number of samples in each STFT window.
        """
        return int(self.window_duration_ms / 1000.0 * self.sample_rate)

    @property
    def hop_length(self) -> int:
        """
        The number of samples between each STFT window.
        """
        return int(self.step_size_ms / 1000.0 * self.sample_rate)


def audio_from_waveform(
    samples: np.ndarray, sample_rate: int, normalize: bool = False
) -> pydub.AudioSegment:
    """
    Convert a numpy array of samples of a waveform to an audio segment.

    Args:
        samples: (channels, samples) array
    """
    # Normalize volume to fit in int16
    if normalize:
        samples *= np.iinfo(np.int16).max / np.max(np.abs(samples))

    # Transpose and convert to int16
    samples = samples.transpose(1, 0)
    samples = samples.astype(np.int16)

    # Write to the bytes of a WAV file
    wav_bytes = io.BytesIO()
    wavfile.write(wav_bytes, sample_rate, samples)
    wav_bytes.seek(0)

    # Read into pydub
    return pydub.AudioSegment.from_wav(wav_bytes)

def apply_filters(segment: pydub.AudioSegment, compression: bool = False) -> pydub.AudioSegment:
    """
    Apply post-processing filters to the audio segment to compress it and
    keep at a -10 dBFS level.
    """

    if compression:
        segment = pydub.effects.normalize(
            segment,
            headroom=0.1,
        )

        segment = segment.apply_gain(-10 - segment.dBFS)

        segment = pydub.effects.compress_dynamic_range(
            segment,
            threshold=-20.0,
            ratio=4.0,
            attack=5.0,
            release=50.0,
        )

    desired_db = -12
    segment = segment.apply_gain(desired_db - segment.dBFS)

    segment = pydub.effects.normalize(
        segment,
        headroom=0.1,
    )

    return segment


class SpectrogramConverter:
    """
    Convert between audio segments and spectrogram tensors using torchaudio.

    In this class a "spectrogram" is defined as a (batch, time, frequency) tensor with float values
    that represent the amplitude of the frequency at that time bucket (in the frequency domain).
    Frequencies are given in the perceptul Mel scale defined by the params. A more specific term
    used in some functions is "mel amplitudes".

    The spectrogram computed from `spectrogram_from_audio` is complex valued, but it only
    returns the amplitude, because the phase is chaotic and hard to learn. The function
    `audio_from_spectrogram` is an approximate inverse of `spectrogram_from_audio`, which
    approximates the phase information using the Griffin-Lim algorithm.

    Each channel in the audio is treated independently, and the spectrogram has a batch dimension
    equal to the number of channels in the input audio segment.

    Both the Griffin Lim algorithm and the Mel scaling process are lossy.

    For more information, see https://pytorch.org/audio/stable/transforms.html
    """

    def __init__(self, params: SpectrogramParams, device: str = "cuda"):
        self.p = params

        self.device = device

        if device.lower().startswith("mps"):
            warnings.warn(
                "WARNING: MPS does not support audio operations, falling back to CPU for them",
                stacklevel=2,
            )
            self.device = "cpu"

        # https://pytorch.org/audio/stable/generated/torchaudio.transforms.Spectrogram.html
        self.spectrogram_func = torchaudio.transforms.Spectrogram(
            n_fft=params.n_fft,
            hop_length=params.hop_length,
            win_length=params.win_length,
            pad=0,
            window_fn=torch.hann_window,
            power=None,
            normalized=False,
            wkwargs=None,
            center=True,
            pad_mode="reflect",
            onesided=True,
        ).to(self.device)

        # https://pytorch.org/audio/stable/generated/torchaudio.transforms.GriffinLim.html
        self.inverse_spectrogram_func = torchaudio.transforms.GriffinLim(
            n_fft=params.n_fft,
            n_iter=params.num_griffin_lim_iters,
            win_length=params.win_length,
            hop_length=params.hop_length,
            window_fn=torch.hann_window,
            power=1.0,
            wkwargs=None,
            momentum=0.99,
            length=None,
            rand_init=True,
        ).to(self.device)

        # https://pytorch.org/audio/stable/generated/torchaudio.transforms.MelScale.html
        self.mel_scaler = torchaudio.transforms.MelScale(
            n_mels=params.num_frequencies,
            sample_rate=params.sample_rate,
            f_min=params.min_frequency,
            f_max=params.max_frequency,
            n_stft=params.n_fft // 2 + 1,
            norm=params.mel_scale_norm,
            mel_scale=params.mel_scale_type,
        ).to(self.device)

        # https://pytorch.org/audio/stable/generated/torchaudio.transforms.InverseMelScale.html
        self.inverse_mel_scaler = torchaudio.transforms.InverseMelScale(
            n_stft=params.n_fft // 2 + 1,
            n_mels=params.num_frequencies,
            sample_rate=params.sample_rate,
            f_min=params.min_frequency,
            f_max=params.max_frequency,
            norm=params.mel_scale_norm,
            mel_scale=params.mel_scale_type,
        ).to(self.device)

    def spectrogram_from_audio(
        self,
        audio: pydub.AudioSegment,
    ) -> np.ndarray:
        """
        Compute a spectrogram from an audio segment.

        Args:
            audio: Audio segment which must match the sample rate of the params

        Returns:
            spectrogram: (channel, frequency, time)
        """
        assert int(audio.frame_rate) == self.p.sample_rate, "Audio sample rate must match params"

        # Get the samples as a numpy array in (batch, samples) shape
        waveform = np.array([c.get_array_of_samples() for c in audio.split_to_mono()])

        # Convert to floats if necessary
        if waveform.dtype != np.float32:
            waveform = waveform.astype(np.float32)

        waveform_tensor = torch.from_numpy(waveform).to(self.device)
        amplitudes_mel = self.mel_amplitudes_from_waveform(waveform_tensor)

        spectrogram = amplitudes_mel.cpu().numpy()

        return spectrogram[0] if spectrogram.shape[0] == 1 else spectrogram

    def audio_from_spectrogram(
        self,
        spectrogram: np.ndarray,
        do_apply_filters: bool = True,
    ) -> pydub.AudioSegment:
        """
        Reconstruct an audio segment from a spectrogram.

        Args:
            spectrogram: (batch, frequency, time)
            apply_filters: Post-process with normalization and compression

        Returns:
            audio: Audio segment with channels equal to the batch dimension
        """

        print("Converting... ", end='', flush=True)

        # Move to device
        amplitudes_mel = torch.from_numpy(spectrogram).to(self.device)

        # Reconstruct the waveform
        waveform = self.waveform_from_mel_amplitudes(amplitudes_mel)

        # Convert to audio segment
        segment = audio_from_waveform(
            samples=waveform.cpu().numpy(),
            sample_rate=self.p.sample_rate,
            # Normalize the waveform to the range [-1, 1]
            normalize=True,
        )

        # Optionally apply post-processing filters
        if do_apply_filters:
            segment = apply_filters(
                segment,
                compression=False,
            )

        print("done.")

        return segment

    def mel_amplitudes_from_waveform(
        self,
        waveform: torch.Tensor,
    ) -> torch.Tensor:
        """
        Torch-only function to compute Mel-scale amplitudes from a waveform.

        Args:
            waveform: (batch, samples)

        Returns:
            amplitudes_mel: (batch, frequency, time)
        """
        # Compute the complex-valued spectrogram
        spectrogram_complex = self.spectrogram_func(waveform)

        # Take the magnitude
        amplitudes = torch.abs(spectrogram_complex)

        # Convert to mel scale
        return self.mel_scaler(amplitudes)

    def waveform_from_mel_amplitudes(
        self,
        amplitudes_mel: torch.Tensor,
    ) -> torch.Tensor:
        """
        Torch-only function to approximately reconstruct a waveform from Mel-scale amplitudes.

        Args:
            amplitudes_mel: (batch, frequency, time)

        Returns:
            waveform: (batch, samples)
        """
        # Convert from mel scale to linear
        amplitudes_linear = self.inverse_mel_scaler(amplitudes_mel)

        # Run the approximate algorithm to compute the phase and recover the waveform
        return self.inverse_spectrogram_func(amplitudes_linear)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Training data

For this task, your training data is a collection of music scraped from the internet, in particular from [The Mod Archive](https://modarchive.org/) and [Amiga Music Preservation](https://amp.dascene.net/). This is old music from the days of the [Demoscene](https://en.wikipedia.org/wiki/Demoscene), and importantly, it's free!

We have scraped the training data for you in mp3 format. **START DOWNLOADING NOW** in the interest of time!

Get it from one of these mirrors (1.8GB): [[1]](https://drive.google.com/file/d/1HZs6l4eSXNqDUAd-SrvQMZYr-VwYVO9z/view?usp=sharing) [[2]](https://drive.google.com/file/d/1JRZE0fYmhm6Rs6O2GK87Ft0yLgH9o9sq/view?usp=sharing) [[3]](https://drive.google.com/file/d/17yp8E1N1PLw2bQ0Jsr3l3IHKbWumesBc/view?usp=sharing) [[4]](https://drive.google.com/file/d/1tTUX0C0awucasjj3vbRvzs6O4_tktLLE/view?usp=sharing) [[5]](https://drive.google.com/file/d/1D_cDR9CCfHYeg-UTFaUdFnoapw3xcUQ1/view?usp=sharing) [[6]](https://drive.google.com/file/d/1C0PoTZVL0i2LFk_b8pTbsQOUi2Jg6Xl9/view?usp=sharing) [[7]](https://drive.google.com/file/d/1-MgDBK_1-EtPAuaKkyTZfGW3efn-T4QO/view?usp=sharing)

Please communicate with other groups in order to use one link per group to avoid bandwidth limitations from Google.

Once you have downloaded the zip file, unzip it into the "mp3" folder and upload this new folder [on the root of your Google Drive](https://drive.google.com/drive/u/0/my-drive)

To start experimenting immediately, [here](https://drive.google.com/file/d/1DZRyHKHWaIl0Qh_ahce9mcyEeLAKId7k/view?usp=sharing)'s a single test track (~2MB).

Once the dataset is downloaded, mount your Google Drive (Colab sidebar → Files → Mount Drive) and test if it works:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# replace with your own path
audio = pydub.AudioSegment.from_mp3("./drive/MyDrive/mp3/0istein Eide - Funkanette.mp3")

audio

### Data representation

Once you have loaded a track, you can access its sequence of audio samples as follows:

In [ ]:
# just cropping 5 seconds of music
chunk = audio[20_000:25_000]

plt.figure(figsize=(7,3))
plt.plot(chunk.get_array_of_samples())
plt.axis('off')
plt.show()

Let's play it:

In [ ]:
chunk

You can work with this representation, and treat it as a sequence of symbols just like you did with text in your name generator during yesterday's interview.

Alternatively, you can work with **spectrograms**.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

params = SpectrogramParams(
            sample_rate=44100,
            stereo=False,
            step_size_ms=10,
            min_frequency=20,
            max_frequency=20000,
            num_frequencies=512,
        )

sc = SpectrogramConverter(params=params, device=device)

For example, let's compute a spectrogram for the five seconds of music from above:

**WARNING:** Don't compute spectrograms for entire songs, it's slow and takes up a lot of RAM!

In [ ]:
chunk = audio[20_000:25_000]
spectrogram = sc.spectrogram_from_audio(chunk)

plt.figure(figsize=(7,4))
plt.imshow(spectrogram, origin='lower', aspect='auto', interpolation="nearest")
plt.ylabel('Freq. bin')
plt.xlabel('Time (ms)')
plt.title('Spectrogram', fontsize=10)
plt.show()

The spectrogram is a 2D (time-frequency) representation for a piece of music.

With this representation, you could think of your music as a sequence of vertical spectrogram slices, of a few milliseconds each.

Therefore, you could generate spectrogram slices, put them together, and then convert back to the audio domain!

Here's some code for converting spectrograms back to audio:

In [ ]:
# this is slow!
chunk_recon = sc.audio_from_spectrogram(spectrogram[None, :], do_apply_filters=False)

chunk_recon

Note that the reconstruction is not perfect, due to quantization parameters in `params`.

We don't care too much about sound quality though.

We care about **training a working music generator!** That's why you were hired in the first place.

### Training a music generator

On to you: show us what you can do!

In [ ]:
# here